In [1]:
from gsheets import Sheets
import os
import pandas as pd
import operator
import mysql.connector
import shutil
from shutil import copyfile
import os.path
import csv
from gspread_pandas import Spread, Client 
from oauth2client.service_account import ServiceAccountCredentials
import pprint
import gspread
import math
import numpy as np
import sys
import time
import random 

In [2]:
#local work
#defining internal lic
lic=[   'df',            
        'e',
        'l',
        'h',
        'n',
        'sh',
        'sl',
        'sla',
        'z',
        'jm',
        'jd',
        'lo',
        'mc',
        'nc',
        'mss',  
        'me',
        'az',
        'hs',
        'nh'
     
    ]
#devices
devices=[
        'l11',
        'l12',
        'l13',
        'n1',
        'n2',
        'n3',
        'n5',
        'n6',
        'n7',
        'd',
        'd2'
        ]

In [3]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
os.chdir('/Users/innovatus6/')
creds = ServiceAccountCredentials.from_json_keyfile_name('linkupdater-631ef6e77556.json', scope)

client = gspread.authorize(creds)

sheet= client.open('Master Data Builds').sheet1

df = pd.DataFrame(sheet.get_all_records())

In [4]:
#extracting rows that has counts and are not done
#number check
numcheck=[0]*len(df.index)

for i in df.index:    
    if df.index[i] in [i for i, x in enumerate(df.number) if isinstance(x, (int, float, complex)) and not isinstance(x, bool) == "TRUE"]:
        numcheck[i] = 1
        
    else: 
        numcheck[i] = 0

#combining the above number check and status check to filter the dataset        
df_extracted=df[operator.and_(operator.and_(operator.and_(df.status!="done",df.status!="this link is building now"),numcheck),df.lic=="")]

#sorting the extracted database by count
df_extracted=df_extracted.reset_index(drop=True)
df_extracted

AttributeError: 'DataFrame' object has no attribute 'status'

In [ ]:
#here we add a very small rn to the column number, so that the numbers themselves can be used as unique identifiers
#this is to deal with the rare case where two or more links have the same number

random.uniform(0,0.0000000000001)

for i in range(len(df_extracted.number)):
    df_extracted.iloc[i,6]=df_extracted.at[i,'number']+random.uniform(0,0.00005)

In [ ]:
def allocator(numbers, targetrange, partial=[]):        
    s=sum(partial)
    #going through each of the numbers as "choosen one" as a means to work through the combinations
    for i in range(len(numbers)):
        n=numbers[i]
        remaining=numbers[i+1:]                        
       # print("pre-frame: remaining is currently: "+str(remaining) + ", partial+n is: "+str(partial+[n]))
        
        #running recursion function until breaking point is reached
        allocator(remaining, targetrange, partial + [n])            
      #  print("previous-frame: remaining is currently: "+str(remaining))
    
    #print("s for the previous frame is: "+ str(s))
    #trying to stopping function once a suitable group has been found
    if targetrange[0]<=s<=targetrange[1] : 
        solutions.append(partial)
        #print ("sum(%s)=%s" % (partial,targetrange))

In [ ]:
start_time = time.time()

for j in range(len(lic)):

    numbers=list(df_extracted.number)[:10]
    targetrange=[1470,1520]
    solutions=[]
    allocator(numbers,targetrange)

    try: 
        print(solutions[0])
    except:
        break

    #printing license and devices to the numbers in the first solution

    for i in range(len(solutions[0])):    
        #extracting links 
        #note: we sleep the so we dont exceed the google api limit
        locallink=df_extracted[df_extracted['number'].isin(solutions[0])].link

        sheet.update_cell(int(df[df.iloc[:,7]==locallink.iloc[i]].index.to_numpy())+2,6, lic[0])
        time.sleep(1)
        sheet.update_cell(int(df[df.iloc[:,7]==locallink.iloc[i]].index.to_numpy())+2,5, devices[0])
        time.sleep(1)
        
    #updating the remaining df
    df_extracted=df_extracted[~df_extracted['number'].isin(solutions[0])]

    #updating the remaining license/devices
    lic=lic[1:]
    devices=(devices*3)[1:]


print("--- %s seconds ---" % (time.time() - start_time))

In [5]:
creds